In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers

print(f"tf.__version__ = {tf.__version__}")
print(f"hub.__version__ = {hub.__version__}")

tf.__version__ = 2.3.0
hub.__version__ = 0.10.0


In [2]:
FEATURE_EXTRACTOR_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4"

CHECK_POINT_DIR = './checkpoint'
INPUT_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
(train_data_ds, val_data_ds), metadata = tfds.load(
    name='tf_flowers',
    split=['train[:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)
num_classes = metadata.features['label'].num_classes
class_names = metadata.features['label'].names
print(f"num_classes = {num_classes}, class_names = {class_names}")

num_classes = 5, class_names = ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']


In [4]:
def resize_and_rescale(image, label):
    image = tf.image.resize(image, [INPUT_SIZE, INPUT_SIZE])
    image = (image / 255.0)
    return image, label

def encode_one_hot(image, label):
    label = tf.one_hot(label, num_classes)
    return image, label

def augment(image, label):
    image = tf.cast(image, tf.float32)
    shape = tf.shape(image)
    height, width = shape[0], shape[1]
    min_size = tf.minimum(height, width)
    crop_size = tf.cast(tf.multiply(tf.cast(min_size, tf.float32), 0.8), tf.int32)
    image = tf.image.random_crop(image, size=[crop_size, crop_size, 3])
    image = tf.image.random_flip_left_right(image)
    return image, label

train_data = (
    train_data_ds
    .shuffle(4096)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .map(encode_one_hot, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
) 

In [5]:
def center_crop(image, label):
    image = tf.cast(image, tf.float32)
    shape = tf.shape(image)
    height, width = shape[0], shape[1]
    size = tf.minimum(height, width)
    image = tf.image.crop_to_bounding_box(image, (height - size) // 2, (width - size) // 2, size, size)
    return image, label
    
val_data = (
    val_data_ds
    .map(center_crop, num_parallel_calls=AUTOTUNE)
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .map(encode_one_hot, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

In [6]:
base_model = keras.applications.MobileNetV2(
    weights='imagenet',
    alpha=1.4,
    include_top=False,
    pooling='max',
    input_shape=(INPUT_SIZE, INPUT_SIZE, 3),
    classes=num_classes
)

In [7]:
model = keras.models.Sequential([
    base_model,
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax'),
])

model.compile(
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1), 
    optimizer=keras.optimizers.Adam(lr=1e-4), 
    metrics=['accuracy']
)

In [8]:
DECAY_START = 3

def scheduler(epoch):
    if epoch < DECAY_START:
        return 1e-4
    else:
        return max(1e-4 * np.exp(0.1 * (DECAY_START - epoch)), 1e-6)

lr_callback = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

In [9]:
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=CHECK_POINT_DIR, 
    save_best_only=True, 
    monitor='val_loss', 
    mode='min', 
    verbose=1
)

In [10]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[lr_callback, checkpoint_callback],
    verbose=1
)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 1/20
  2/104 [..............................] - ETA: 13s - loss: 5.9630 - accuracy: 0.2656WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0672s vs `on_train_batch_end` time: 0.1933s). Check your callbacks.


104/104 [==============================] - ETA: 0s - loss: 2.8973 - accuracy: 0.6452
Epoch 00001: val_loss improved from inf to 1.75571, saving model to ./checkpoint
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


...

104/104 [==============================] - 48s 466ms/step - loss: 0.5885 - accuracy: 0.9503 - val_loss: 0.5917 - val_accuracy: 0.9755

Epoch 00019: LearningRateScheduler reducing learning rate to 2.2313016014842984e-05.
Epoch 19/20
104/104 [==============================] - ETA: 0s - loss: 0.5882 - accuracy: 0.9503
Epoch 00019: val_loss improved from 0.59166 to 0.57697, saving model to ./checkpoint
INFO:tensorflow:Assets written to: ./checkpoint/assets


INFO:tensorflow:Assets written to: ./checkpoint/assets


104/104 [==============================] - 48s 466ms/step - loss: 0.5882 - accuracy: 0.9503 - val_loss: 0.5770 - val_accuracy: 0.9673

Epoch 00020: LearningRateScheduler reducing learning rate to 2.018965179946554e-05.
Epoch 20/20
104/104 [==============================] - ETA: 0s - loss: 0.5768 - accuracy: 0.9609
Epoch 00020: val_loss improved from 0.57697 to 0.56561, saving model to ./checkpoint
INFO:tensorflow:Assets written to: ./checkpoint/assets


INFO:tensorflow:Assets written to: ./checkpoint/assets


104/104 [==============================] - 48s 465ms/step - loss: 0.5768 - accuracy: 0.9609 - val_loss: 0.5656 - val_accuracy: 0.9700
